Author: Brady Wilkinson<br>
Date: 19 September 2022
<br>
<br>
<br>
<br>

# ACLED API User Guide:
https://acleddata.com/acleddatanew/wp-content/uploads/2020/10/ACLED_API-User-Guide_2020.pdf
<br>
<br>
<br>
<br>
## There are two main functions in this workbook: `get_acled_dataframe` and `get_url`.

### `get_acled_dataframe`
Takes a URL string as a parameter and returns a Pandas dataframe if the URL fits the criteria for an ACLED API data request. You can either provide your own URL string, or you can get the URL string from the `get_url` function (below).
<br>
### `get_url`
Takes several optional parameters of filters for the desired data from ACLED, and generates the URL formatted with the filters to pass into `get_acled_dataframe`. Some of the optional parameters have to be in a specific format. For example, you can't just specify a region name for the `region` parameter. Instead, ACLED provides unique IDs for each region, and the function must take the integer representation of the region.<br>
(Note: this function uses Brady's email and private key specially given by ACLED to let the account make unlimited data requests. Please be careful when using.)

In [1]:
import pandas as pd

import csv
import requests
import json


def get_acled_dataframe(url):

    with requests.Session() as s:
        download = s.get(url)

    decoded_content = download.content.decode('utf-8')

    res = json.loads(decoded_content)
    data = res['data']

    return pd.DataFrame(data)

In [2]:
# THESE PARAMETERS MUST BE IN A SPECIFIC FORMAT
    # limit: integer of desired row limit in dataframe
    # date (str): YYYY-MM-DD
    # date_range (str): YYYY-MM-DD:AND:YYYY-MM-DD
    # region (int): Numeric ID for desired region

# THESE PARAMETERS USE 'LIKE' IN ACLED REQUEST BACKEND, SO THEY DO NOT HAVE TO BE EXACT.
    # source (str)
    # source_scale (str)
    # event_type (str)
    # notes (str)

def get_url(
    limit=100,
    date=None,
    date_range=None,
    region=None,
    source=None,
    source_scale=None,
    event_type=None,
    notes=None
):
   
    email = 'bwilki13@gmu.edu'
    acled_key = 'fTIpbk9YOV5PYNcjf3v3'
    
    url = f'https://api.acleddata.com/acled/read?key={acled_key}&email={email}'
    
    filters = f'&limit={limit}'
    if date:
        filters += f'&event_date={date}'
    if date_range:
        filters += f'&event_date:BETWEEN:{date_range}'
    if region:
        filters += f'&region={region}'
    if source:
        filters += f'&source={source}'
    if source_scale:
        filters += f'&source_scale={source_scale}'
    if event_type:
        filters += f'&event_type={event_type}'
    if notes:
        filters += f'&notes={notes}'
    
    return url + filters

In [3]:
# Example

url = get_url(limit=100, region=7)
df = get_acled_dataframe(url)
df.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,9491367,356,IND118474,118474,2022-09-16,2022,1,Protests,Peaceful protest,Protesters (India),...,Poonch,33.7703,74.0925,1,Amar Ujala,National,"On 16 September 2022, workers of the water dep...",0,1663689747,IND
1,9491372,356,IND118550,118550,2022-09-16,2022,1,Protests,Peaceful protest,Protesters (India),...,Kathua,32.3867,75.5174,1,Amar Ujala,National,"On 16 September 2022, locals held a protest at...",0,1663689747,IND
2,9491373,356,IND118574,118574,2022-09-16,2022,1,Explosions/Remote violence,Remote explosive/landmine/IED,Kashmir Rebels (India),...,Fazal Abad,33.6086,74.3245,1,Amar Ujala,National,"On 16 September 2022, militants (coded as Kash...",0,1663689747,IND
3,9491387,356,IND118229,118229,2022-09-16,2022,1,Protests,Peaceful protest,Protesters (India),...,Chengalpattu,12.6918,79.9766,1,Asian News International,National,"On 16 September 2022, members of AIADMK held a...",0,1663689747,IND
4,9491391,356,IND118183,118183,2022-09-16,2022,1,Riots,Mob violence,Rioters (India),...,Kollam,8.8811,76.5846,1,Asian News International,National,"On 16 September 2022, three members of Kerala ...",0,1663689747,IND


``` 
 key = {api_key}
 email = {email address associated with key}
 data_id = ?data_id={number}
 iso = ?iso={number}
 event_id_cnty LIKE ?event_id_cnty={text}
 event_id_no_cnty LIKE ?event_id_no_cnty={text}
 event_date = ?event_date={yyyy-mm-dd}
 year = ?year={yyyy}
 time_precision = ?time_precision={number}
 event_type LIKE ?event_type={text}
 sub_event_type LIKE ?sub_event_type={text}
 actor1 LIKE ?actor1={text}
 assoc_actor_1 LIKE ?assoc_actor_1={text}
 inter1 = ?inter1={number}
 actor2 LIKE ?actor2={text}
 assoc_actor_2 LIKE ?assoc_actor_2={text}
 inter2 = ?inter2={number}
 interaction = ?interaction={number}
 region = ?region={number}
 country LIKE ?country={text}
 admin1 LIKE ?admin1={text}
 admin2 LIKE ?admin2={text}
 admin3 LIKE ?admin3={text}
 location LIKE ?location={text}
 latitude = ?latitude={number}
 longitude = ?longitude={number}
 geo_precision = ?geo_precision={number}
 source LIKE ?source={text}
 source_scale LIKE ?source_scale={text}
 notes LIKE ?notes={text}
 fatalities = ?fatalities={number}
 timestamp >= ?timestamp={number/yyyy-mm-dd}
 export_type = ?export_type={text}
 iso3 = ?iso3={text}
```

In [4]:
# Western Africa 1
# Middle Africa 2
# Eastern Africa 3
# Southern Africa 4
# Northern Africa 5
# South Asia 7
# Western Asia 8
# Southeast Asia 9
# Middle East 11
# Europe 12
# Caucasus and Central Asia 13
# Central America 14
# South America 15
# Caribbean 16
# East Asia 17